In [1]:
import numpy as np, sys, os, holoviews as hv, scipy
sys.path.insert(0, '../') # add normal modules
from Utils import IO, plotting as plotz

root = '../Data/new5/2019-06-04T19:13:18.214148'
data = IO.DataLoader(root)

settings = {k: IO.Settings(os.path.join(root, k)) for k in data}

import multiprocessing as mp
from multiprocessing.pool import ThreadPool
processes = mp.cpu_count()

aucs = loadedData = {}
for k, v in data.items():
    tmp = os.path.join(root, k)
    setting = settings[k]
    
    tmp_worker = IO.Worker(v, settings[k])
    with ThreadPool(processes = processes) as p:
        p.map(tmp_worker, tmp_worker.idx)
        v = np.frombuffer(tmp_worker.buff, dtype = np.float64).reshape(*tmp_worker.buffshape)    
    del tmp_worker
    s = v.shape
    v = v.reshape(-1, setting.nNodes, setting.deltas // 2 - 1)
    v = np.array([(i - i.min()) / (i.max() - i.min()) for i in v])
    v = v.reshape(-1, s[-1])
    loadedData[k] = v.reshape(s)
    
    




  0%|          | 0/39 [00:00<?, ?it/s]

Extracting data...
Done
Reading settings in ../Data/new5/2019-06-04T19:13:18.214148/2019-06-04T19:36:18.654406: using json
loading graph
Reading settings in ../Data/new5/2019-06-04T19:13:18.214148/2019-06-04T21:34:47.884099: using json
loading graph
Reading settings in ../Data/new5/2019-06-04T19:13:18.214148/2019-06-04T20:01:12.258815: using json
loading graph
Reading settings in ../Data/new5/2019-06-04T19:13:18.214148/2019-06-04T20:33:36.795955: using json
loading graph
Reading settings in ../Data/new5/2019-06-04T19:13:18.214148/2019-06-04T20:13:28.018046: using json
loading graph
Reading settings in ../Data/new5/2019-06-04T19:13:18.214148/2019-06-04T19:17:36.597872: using json
loading graph
Reading settings in ../Data/new5/2019-06-04T19:13:18.214148/2019-06-04T20:18:42.659934: using json
loading graph
Reading settings in ../Data/new5/2019-06-04T19:13:18.214148/2019-06-04T20:53:46.257622: using json
loading graph
Reading settings in ../Data/new5/2019-06-04T19:13:18.214148/2019-06-04T2

100%|██████████| 39/39 [00:28<00:00,  3.69s/it]
0it [00:00, ?it/s]/home/casper/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in true_divide

100%|██████████| 39/39 [00:34<00:00,  4.48s/it]

  0%|          | 0/41 [00:00<?, ?it/s]

  2%|▏         | 1/41 [00:01<01:10,  1.77s/it]

  7%|▋         | 3/41 [00:01<00:34,  1.09it/s]

 12%|█▏        | 5/41 [00:02<00:17,  2.02it/s]

 15%|█▍        | 6/41 [00:02<00:15,  2.28it/s]

 17%|█▋        | 7/41 [00:02<00:14,  2.38it/s]

 17%|█▋        | 7/41 [00:02<00:14,  2.38it/s]

 17%|█▋        | 7/41 [00:02<00:14,  2.38it/s]

 20%|█▉        | 8/41 [00:04<00:26,  1.24it/s]

 22%|██▏       | 9/41 [00:04<00:19,  1.65it/s]

 27%|██▋       | 11/41 [00:04<00:13,  2.22it/s]

 32%|███▏      | 13/41 [00:04<00:07,  3.50it/s]

 34%|███▍      | 14/41 [00:04<00:06,  4.11it/s]

 34%|███▍      | 14/41 [00:04<00:06,  4.11it/s]

 37%|███▋      | 15/41 [00:05<00:12,  2.07it/s]

 39%|███▉      | 16/41 [00:06<00

 73%|███████▎  | 30/41 [00:09<00:04,  2.32it/s]



 78%|███████▊  | 32/41 [00:09<00:03,  2.97it/s]



 80%|████████  | 33/41 [00:10<00:02,  3.05it/s]



 83%|████████▎ | 34/41 [00:10<00:01,  3.66it/s]



 85%|████████▌ | 35/41 [00:10<00:02,  2.59it/s]



 88%|████████▊ | 36/41 [00:11<00:01,  3.08it/s]



 90%|█████████ | 37/41 [00:11<00:01,  3.89it/s]



 93%|█████████▎| 38/41 [00:11<00:00,  4.88it/s]



 95%|█████████▌| 39/41 [00:11<00:00,  4.53it/s]



 98%|█████████▊| 40/41 [00:27<00:05,  5.05s/it]




  0%|          | 0/41 [00:00<?, ?it/s]




  2%|▏         | 1/41 [00:01<00:57,  1.45s/it]




 17%|█▋        | 7/41 [00:01<00:18,  1.86it/s]




 17%|█▋        | 7/41 [00:01<00:18,  1.86it/s]




 17%|█▋        | 7/41 [00:01<00:18,  1.86it/s]




 17%|█▋        | 7/41 [00:01<00:18,  1.86it/s]




 17%|█▋        | 7/41 [00:01<00:18,  1.86it/s]




 20%|█▉        | 8/41 [00:03<00:27,  1.19it/s]




 22%|██▏       | 9/41 [00:03<00:20,  1.57it/s]




 24%|██▍       | 10/41 [00:03<00:15,  

 76%|███████▌  | 31/41 [00:08<00:02,  3.70it/s]






 78%|███████▊  | 32/41 [00:08<00:02,  4.05it/s]






 83%|████████▎ | 34/41 [00:09<00:02,  3.47it/s]






 83%|████████▎ | 34/41 [00:09<00:02,  3.47it/s]






 85%|████████▌ | 35/41 [00:09<00:01,  3.14it/s]






 88%|████████▊ | 36/41 [00:09<00:01,  3.74it/s]






 90%|█████████ | 37/41 [00:09<00:00,  4.03it/s]






 93%|█████████▎| 38/41 [00:09<00:00,  4.83it/s]






 98%|█████████▊| 40/41 [00:26<00:02,  2.66s/it]







  0%|          | 0/41 [00:00<?, ?it/s]







  2%|▏         | 1/41 [00:02<01:23,  2.10s/it]







 10%|▉         | 4/41 [00:02<00:44,  1.19s/it]







 12%|█▏        | 5/41 [00:02<00:31,  1.15it/s]







 12%|█▏        | 5/41 [00:02<00:31,  1.15it/s]







 15%|█▍        | 6/41 [00:02<00:24,  1.45it/s]







 15%|█▍        | 6/41 [00:02<00:24,  1.45it/s]







 17%|█▋        | 7/41 [00:02<00:21,  1.56it/s]







 20%|█▉        | 8/41 [00:04<00:26,  1.26it/s]







 22%|██▏       | 9/41 [00:04<00:20,

...

 22%|██▏       | 8/37 [00:03<00:16,  1.80it/s]


















 24%|██▍       | 9/37 [00:03<00:14,  1.98it/s]


















 30%|██▉       | 11/37 [00:03<00:09,  2.62it/s]


















 32%|███▏      | 12/37 [00:04<00:10,  2.33it/s]


















 35%|███▌      | 13/37 [00:04<00:08,  2.88it/s]


















 38%|███▊      | 14/37 [00:04<00:06,  3.46it/s]


















 41%|████      | 15/37 [00:04<00:06,  3.22it/s]


















 43%|████▎     | 16/37 [00:05<00:06,  3.26it/s]


















 46%|████▌     | 17/37 [00:05<00:05,  4.00it/s]


















 49%|████▊     | 18/37 [00:05<00:06,  2.86it/s]


















 51%|█████▏    | 19/37 [00:05<00:06,  2.94it/s]


















 54%|█████▍    | 20/37 [00:06<00:04,  3.42it/s]


















 57%|█████▋    | 21/37 [00:06<00:05,  2.91it/s]


















 62%|██████▏   | 23/37 [00:07<00:05,  2.51it/s]


















 65%|██████▍   | 24/37 [00:07<00:04,  2.87it/s]


















 65%|██████▍

  7%|▋         | 3/41 [00:02<00:37,  1.02it/s]




















 10%|▉         | 4/41 [00:02<00:26,  1.39it/s]




















 12%|█▏        | 5/41 [00:02<00:19,  1.85it/s]




















 15%|█▍        | 6/41 [00:02<00:14,  2.49it/s]




















 17%|█▋        | 7/41 [00:02<00:12,  2.65it/s]




















 20%|█▉        | 8/41 [00:04<00:24,  1.33it/s]




















 22%|██▏       | 9/41 [00:04<00:21,  1.51it/s]




















 24%|██▍       | 10/41 [00:04<00:16,  1.86it/s]




















 32%|███▏      | 13/41 [00:05<00:10,  2.62it/s]




















 32%|███▏      | 13/41 [00:05<00:10,  2.62it/s]




















 32%|███▏      | 13/41 [00:05<00:10,  2.62it/s]




















 34%|███▍      | 14/41 [00:05<00:09,  2.79it/s]




















 37%|███▋      | 15/41 [00:06<00:13,  1.88it/s]




















 39%|███▉      | 16/41 [00:06<00:11,  2.25it/s]




















 41%|████▏     | 17/41 [00:07<00:08,  2.67it/s]









 66%|██████▌   | 27/41 [00:10<00:05,  2.74it/s]





















 68%|██████▊   | 28/41 [00:11<00:07,  1.76it/s]





















 71%|███████   | 29/41 [00:12<00:05,  2.05it/s]





















 76%|███████▌  | 31/41 [00:12<00:03,  2.52it/s]





















 80%|████████  | 33/41 [00:12<00:02,  3.27it/s]





















 80%|████████  | 33/41 [00:12<00:02,  3.27it/s]





















 83%|████████▎ | 34/41 [00:13<00:02,  3.42it/s]





















 85%|████████▌ | 35/41 [00:13<00:02,  2.24it/s]





















 88%|████████▊ | 36/41 [00:14<00:02,  2.33it/s]





















 90%|█████████ | 37/41 [00:14<00:01,  2.70it/s]





















 93%|█████████▎| 38/41 [00:14<00:01,  2.85it/s]





















 95%|█████████▌| 39/41 [00:14<00:00,  3.42it/s]





















 98%|█████████▊| 40/41 [00:39<00:07,  7.70s/it]





















100%|██████████| 41/41 [00:39<00:00,  5.42s/it]






















  0%|          | 0/41 [00:00<?, ?

 17%|█▋        | 7/41 [00:02<00:12,  2.76it/s]























 20%|█▉        | 8/41 [00:03<00:19,  1.73it/s]























 27%|██▋       | 11/41 [00:03<00:13,  2.27it/s]























 29%|██▉       | 12/41 [00:03<00:10,  2.74it/s]























 32%|███▏      | 13/41 [00:04<00:10,  2.75it/s]























 34%|███▍      | 14/41 [00:04<00:08,  3.24it/s]























 37%|███▋      | 15/41 [00:05<00:12,  2.12it/s]























 41%|████▏     | 17/41 [00:05<00:08,  2.72it/s]























 41%|████▏     | 17/41 [00:05<00:08,  2.72it/s]























 44%|████▍     | 18/41 [00:05<00:06,  3.30it/s]























 46%|████▋     | 19/41 [00:05<00:05,  3.68it/s]























 49%|████▉     | 20/41 [00:05<00:04,  4.24it/s]























 51%|█████     | 21/41 [00:05<00:04,  4.66it/s]























 54%|█████▎    | 22/41 [00:06<00:08,  2.25it/s]























 56%|███

 86%|████████▌ | 30/35 [00:08<00:01,  3.27it/s]
























 89%|████████▊ | 31/35 [00:08<00:00,  4.01it/s]
























 94%|█████████▍| 33/35 [00:09<00:00,  3.19it/s]
























 97%|█████████▋| 34/35 [00:23<00:04,  4.41s/it]

In [2]:
# a = next(iter(data))
# data[a]['0.8']['1']

In [3]:
def worker(sample):
    # tmp workaround
    if len(sample.shape) == 1:
        sample = sample.reshape(-1, 1)
    auc = np.zeros((len(sample), 2))
    coeffs, errors = plotz.fit(sample, func, params = fitParam)
    for nodei, c in enumerate(coeffs):
        tmp = 0
        F      = lambda x: func(x, *c)
        tmp, _ = scipy.integrate.quad(F, 0, LIMIT)
        auc[nodei, 0] = tmp
        auc[nodei, 1] = errors[nodei]
        if errors[nodei] > .1:
            print('error large')
    auc[auc < np.finfo(auc.dtype).eps ] = 0
    return auc[:, 0]
from tqdm import tqdm

double = lambda x, a, b, c, d, e, f: a + b * np.exp(-c*(x)) + d * np.exp(- e * (x-f))

double_= lambda x, b, c, d, e, f: b * np.exp(-c*(x)) + d * np.exp(- e * (x - f ))
single = lambda x, a, b, c : a + b * np.exp(-c * x)
single_= lambda x, a, b, c : a + b * np.exp(-c * x)
special= lambda x, a, b, c, d: a  + b * np.exp(- (x)**c - d)

func        = double
p0          = np.ones((func.__code__.co_argcount - 1)); # p0[0] = 0
fitParam    = dict(maxfev = int(1e6), \
                   bounds = (0, np.inf), p0 = p0,\
                   jac = 'cs')
aucs = {}
for k, v in tqdm(loadedData.items()):
    setting = settings[k]
    LIMIT = setting.deltas // 2 - 1
#     LIMIT = np.inf
    s = v.shape
    v = v.reshape(-1, *s[-2:])
    try:
        with mp.Pool(mp.cpu_count()) as p:
            auc = np.asarray(p.map(worker, v)).reshape(s[:-1])
        aucs[k] = auc
    except:
        continue



























  0%|          | 0/27 [00:00<?, ?it/s]

























  4%|▎         | 1/27 [00:01<00:37,  1.44s/it]

























  7%|▋         | 2/27 [00:01<00:28,  1.16s/it]

























 11%|█         | 3/27 [00:03<00:30,  1.28s/it]

























 15%|█▍        | 4/27 [00:05<00:30,  1.35s/it]

























 19%|█▊        | 5/27 [00:06<00:30,  1.39s/it]

























 22%|██▏       | 6/27 [00:08<00:30,  1.43s/it]

























 26%|██▌       | 7/27 [00:09<00:29,  1.47s/it]

























 30%|██▉       | 8/27 [00:10<00:26,  1.41s/it]

























 33%|███▎      | 9/27 [00:12<00:26,  1.49s/it]

























 37%|███▋      | 10/27 [00:13<00:24,  1.44s/it]

























 41%|████      | 11/27 [00:15<00:23,  1.46s/it]

























 44%|████▍     | 12/27 [00:16<00:20,  1.36s/it]

























 48%|████▊     | 13/27 [00:17<00:19,  1.39s

In [4]:
from bokeh import plotting as bp
from bokeh.models import ColumnDataSource, CustomJS
import bokeh
bp.output_notebook()


tmp_aucs = np.asarray([i for i in aucs.values()]).squeeze()
print(tmp_aucs.shape)
idx      = np.argmax(tmp_aucs, axis = -1)[:, 0]
x = []
for jdx, i in enumerate(idx):
    tmp = tmp_aucs[jdx, :2, i]
    x.append(tmp)
tmp_aucs = np.asarray(x)
# print(tmp_aucs.shape, idx.shape); assert 0

tmp_mi   = np.asarray([loadedData[i] for i in aucs.keys()]).squeeze()
tmp_mi   = tmp_mi

# setup data
buffer          = ColumnDataSource('x', 'y', \
                                   data = dict(x = [], \
                                               y = [], \
                                               color = [],\
                                              )\
                                  )
buffer_causal   = ColumnDataSource('x', 'y', \
                                   data = dict(x = [], \
                                               y = [],\
                                              )
                                  )

aucs_bokeh = ColumnDataSource('x', 'y', data = dict(x = tmp_aucs[:, 0], \
                                                    y = tmp_aucs[:, 1], \
                                                    color = bokeh.palettes.Viridis256 )\
                             )
mi_bokeh    = ColumnDataSource(data = dict(y = tmp_mi[:, 0].tolist()))
causal_bokeh= ColumnDataSource(data = dict(y = tmp_mi[:, 1].tolist()))

callback = """
//shorthands
var buff_ = buffer.data;
var mi_   = mi.data; 
var caus_ = causal.data;
var buffc_= buffer_causal.data;

//buffers
var x = [];
var y;

var indices = cb_obj.indices;
var nIndex = indices.length;

var tmp, nNodes, nTime;
var idx;

buff_['x'] = [];
buff_['y'] = [];


buffc_['x'] = [];
buffc_['y'] = [];

buff_['color'] = [];

var plt = Bokeh.Plotting;

console.log('start debug')
console.log(Bokeh);
var colors, color;
for (var ni = 0 ; ni < nIndex ; ni++){
    // get the data; build the time vector
    idx      = indices[ni];
    nNodes   = mi_['y'][idx].length;
    
    colors = [];
    for (var node = 0 ; node < nNodes ; node++){
            
        // color = Bokeh.Plotting.color(50+2*1, 100, 150);
        // colors.push(color)
        
        y     = mi_['y'][idx][node];
        nTime = y.length;
        // build time
        x = [];
        for (var t = 0 ; t < nTime ; t++){
            x.push(t);
        }
        buff_['x'].push(x);
        buff_['y'].push(y);
        // buff_['color'].push(colors)
        
        buffc_['x'].push(x);
        buffc_['y'].push(caus_['y'][idx][node]);
        }

}
buffer.change.emit();
buffer_causal.change.emit();
"""


f1 = bp.figure(tools = 'hover tap lasso_select'.split(), width = 300)
f2 = bp.figure( height = int(f1.plot_height * .5 ))
f3 = bp.figure( height = int(f1.plot_height * .5 ))

f1.scatter('x', 'y', source = aucs_bokeh, size = 10)
f2.multi_line('x', 'y', source = buffer)
f3.multi_line('x', 'y', source = buffer_causal)
JS =  CustomJS(\
             args = dict(buffer = buffer, \
                         mi     = mi_bokeh, \
                         buffer_causal = buffer_causal, \
                         causal = causal_bokeh),\
             code = callback)
aucs_bokeh.selected.js_on_change('indices', \
                                JS)
bp.show(bokeh.layouts.row(f1, bokeh.layouts.column(f2, f3)))


Loading BokehJS ...

ValueError: could not broadcast input array from shape (3,1,1,19) into shape (3,1,1)

In [ ]:
# tmp_aucs = np.asarray([i for i in aucs.values()]).squeeze()

# tmp_aucs = np.sort(tmp_aucs, axis = -1)[..., -1:]

# print(tmp_aucs.shape)
# fig, ax = plt.subplots()
# colors = plt.cm.Spectral(np.linspace(0, 1, len(tmp_aucs)))
# [ax.scatter(*i[:2], color = colors[idx]) for idx, i in enumerate(tmp_aucs)]
# fig.show()


for k, v in aucs.items(): 
    print(v.shape)

In [ ]:
plt.cm.Spectral?

In [ ]:
from matplotlib import style
import matplotlib.pyplot as plt
style.reload_library()
%matplotlib inline
style.use('seaborn-poster')
fig, ax = plt.subplots(1,2)
labels = 'Underwhelming Overwhelming'.split()
for k, v in aucs.items():
    tmp = v.squeeze()
    jdx = np.argsort(tmp, axis = -1)[:, -1]
#     print([tmp[i, j] for i, j in enumerate(jdx)])
    for idx, axi in enumerate(ax):
        y = tmp[[0, idx + 1]]
        axi.scatter(*y[:, jdx[idx + 1]], color = 'blue', alpha = .2)
        axi.scatter(*tmp[[0, idx + 1]].max(-1), color = 'red', alpha = .2)
        axi.set_title(labels[idx])
        axi.set(xlabel = 'Informational impact', ylabel = 'Causal impact')
fig.subplots_adjust(wspace = .3)

axi.legend(handles = [plt.Line2D([0], [0], color = i, label = j) for i, j in zip(\
                    'blue red'.split(), 'matched max'.split())], bbox_to_anchor = (1.15, 1))
fig.show()

In [ ]:
import networkx as nx
perc = np.zeros((len(aucs), 2))

# fig, ax = plt.subplots(3, 1, figsize = (10, 16))
colors = plt.cm.gist_ncar(np.linspace(0, 1, len(aucs)))

labels = "Informational impact\tUnderwhelming causal".split('\t')
for idx, (k, v) in enumerate(aucs.items()):
    
    tmp = np.argsort(v.squeeze(), axis = -1)[:, -1]
    perc[idx] = tmp[[0]] == tmp[[1, 2]]
    if not perc[idx, 0]:
        d = loadedData[k].squeeze()
        gs = dict(\
                 width_ratios = [1, 1, 1]\
                 )
        fig, ax = plt.subplots(1,3, gridspec_kw = gs)
        
        mainax = fig.add_subplot(111, \
                 frameon = 0, xticks = [], yticks = [])
        mainax.set_title(k, pad = 25)
        colors = plt.cm.Spectral(np.linspace(0, 1, settings[k].nNodes))
        g = settings[k].graph
        g = nx.node_link_graph(g)
        pos    = nx.circular_layout(g)
        subset = np.argsort(v.squeeze(), -1)[:, -2:]
        print(subset)
        for kk, axi in enumerate(ax[:2]):
            sub = subset[kk]
            [axi.plot(i, color = c, alpha = .4) for i, c in zip(d[kk, sub], colors[sub])]
            axi.set_title(labels[kk])
        nx.draw_networkx_nodes(g, pos = pos, node_color = colors, ax = ax[-1])
        nx.draw_networkx_edges(g, pos = pos, ax = ax[-1])
        ax[-1].set_aspect('equal', 'box')
        ax[-1].axes.axis('off')
        
            
        
#     vv = loadedData[k]
#     for jdx, label in enumerate(tmp):
#         axi = ax[jdx]
#         d = loadedData[k].squeeze()
#         print(d.shape)
#         axi.scatter(*d[[0, jdx + 1], label, :])
#         axi.plot(vv.squeeze()[jdx, label, :])
        
    
print(perc.mean(0), perc.std(0))


In [ ]:
fig, ax = plt.subplots()
for k, v in aucs.items():
    ax.scatter(*v[[0,1]].squeeze())
fig.savefig('test.png')

import networkx as nx
nx.from_pan
nx.from_edgelist([[0,1], [1,0]])